In [1]:
import cmeutils
import cmeutils.polymers as polymer
from cmeutils.gsd_utils import snap_molecule_cluster

## Example

This example is for a .gsd file that contains 3 molecuels, each one with 10 monomers
The molecules are united-atom PEEK, so each monomer contains 22 atoms

In [2]:
gsdfile = "3_10mers.gsd"
peek_system = polymer.System(gsd_file=gsdfile, atoms_per_monomer=22)

/home/chris/miniconda3/envs/cmeutils/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
peek_system.molecules

/home/chris/miniconda3/envs/cmeutils/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
for mol in peek_system.molecules:
    print(mol.center_of_mass)

[-35.71385574 -35.15084457 -28.71744537]
[ 35.79333496   0.57994461 -32.06600189]
[-35.72400284 -26.48761559 -10.47525024]


/home/chris/miniconda3/envs/cmeutils/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
mol_1 = peek_system.molecules[0]
mol_1.monomers

/home/chris/miniconda3/envs/cmeutils/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
for mon in mol_1.monomers:
    print(mon.center_of_mass)

[-35.57025146 -36.42874527 -15.91887856]
[-35.89908218  35.08745575 -19.42394066]
[-35.80141068  34.84962463 -23.63430214]
[-35.84141922  34.34966278 -27.80606079]
[-35.78558731  33.56804657 -31.55745125]
[-35.83119202 -37.00401306 -33.86655426]
[-35.72114182 -33.79898834 -36.84399414]
[-35.73588562 -29.75857925 -35.97425842]
[-35.54290771 -28.27431297 -32.05659103]
[-35.40942001 -27.08843613 -27.9095192 ]


/home/chris/miniconda3/envs/cmeutils/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
